# Transfer Learning CIFAR10

* Train a simple convnet on the CIFAR dataset the first 5 output classes [0..4].
* Freeze convolutional layers and fine-tune dense layers for the last 5 ouput classes [5..9].


### 1. Import CIFAR10 data and create 2 datasets with one dataset having classes from 0 to 4 and other having classes from 5 to 9 

In [0]:
from keras.datasets import cifar10
import pandas as pd
(X_train, y_train), (X_test, y_test)= cifar10.load_data()

In [13]:
print("Number of training examples :", X_train.shape[0], "and each image is of shape (%d, %d)"%(X_train.shape[1], X_train.shape[2]))
print("Number of training examples :", X_test.shape[0], "and each image is of shape (%d, %d)"%(X_test.shape[1], X_test.shape[2]))

Number of training examples : 50000 and each image is of shape (32, 32)
Number of training examples : 10000 and each image is of shape (32, 32)


In [0]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

### 2. Use One-hot encoding to divide y_train and y_test into required no of output classes

In [0]:
from keras.utils import np_utils
num_classes = 10
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)


In [0]:
X_train = X_train/255
X_test = X_test/255

In [0]:
from keras.models import Sequential 
from keras.layers import Dense, Activation 
import keras
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [0]:
input_dim = (32,32,3)

In [19]:
X_test.shape

(10000, 32, 32, 3)

### 3. Build a sequential neural network model which can classify the classes 0 to 4 of CIFAR10 dataset with at least 80% accuracy on test data

In [21]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_dim))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=32,
          epochs=12,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

W0905 17:33:24.695829 140349887555456 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 50000 samples, validate on 10000 samples
Epoch 1/12
50000/50000 [==============================] - 214s 4ms/step - loss: 1.5537 - acc: 0.4433 - val_loss: 1.4195 - val_acc: 0.5149
Epoch 2/12
50000/50000 [==============================] - 211s 4ms/step - loss: 1.1642 - acc: 0.5935 - val_loss: 1.0118 - val_acc: 0.6469
Epoch 3/12
50000/50000 [==============================] - 210s 4ms/step - loss: 1.0281 - acc: 0.6410 - val_loss: 1.0003 - val_acc: 0.6661
Epoch 4/12
50000/50000 [==============================] - 211s 4ms/step - loss: 0.9515 - acc: 0.6735 - val_loss: 0.9960 - val_acc: 0.6590
Epoch 5/12
50000/50000 [==============================] - 210s 4ms/step - loss: 0.9002 - acc: 0.6927 - val_loss: 0.9154 - val_acc: 0.6816
Epoch 6/12
50000/50000 [==============================] - 209s 4ms/step - loss: 0.8763 - acc: 0.7058 - val_loss: 0.9111 - val_acc: 0.7016
Epoch 7/12
50000/50000 [==============================] - 207s 4ms/step - loss: 0.8473 - acc: 0.7161 - val_loss: 0.9574 - 

### 4. In the model which was built above (for classification of classes 0-4 in CIFAR10), make only the dense layers to be trainable and conv layers to be non-trainable

### 5. Utilize the the model trained on CIFAR 10 (classes 0 to 4) to classify the classes 5 to 9 of CIFAR 10  (Use Transfer Learning) <br>
Achieve an accuracy of more than 85% on test data

# Text classification using TF-IDF

### 6. Load the dataset from sklearn.datasets

In [0]:
from sklearn.datasets import fetch_20newsgroups

In [0]:
categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

### 7. Training data

In [4]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

I0905 16:44:05.540847 140404324300672 twenty_newsgroups.py:247] Downloading 20news dataset. This may take a few minutes.
I0905 16:44:05.545768 140404324300672 twenty_newsgroups.py:80] Downloading dataset from https://ndownloader.figshare.com/files/5975967 (14 MB)


### 8. Test data

In [0]:
twenty_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

###  a.  You can access the values for the target variable using .target attribute 
###  b. You can access the name of the class in the target variable with .target_names


In [6]:
twenty_train.target

array([1, 1, 3, ..., 2, 2, 2])

In [7]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [8]:
twenty_train.data[0:5]

['From: sd345@city.ac.uk (Michael Collier)\nSubject: Converting images to HP LaserJet III?\nNntp-Posting-Host: hampton\nOrganization: The City University\nLines: 14\n\nDoes anyone know of a good way (standard PC application/PD utility) to\nconvert tif/img/tga files into LaserJet III format.  We would also like to\ndo the same, converting to HPGL (HP plotter) files.\n\nPlease email any response.\n\nIs this the correct group?\n\nThanks in advance.  Michael.\n-- \nMichael Collier (Programmer)                 The Computer Unit,\nEmail: M.P.Collier@uk.ac.city                The City University,\nTel: 071 477-8000 x3769                      London,\nFax: 071 477-8565                            EC1V 0HB.\n',
 "From: ani@ms.uky.edu (Aniruddha B. Deglurkar)\nSubject: help: Splitting a trimming region along a mesh \nOrganization: University Of Kentucky, Dept. of Math Sciences\nLines: 28\n\n\n\n\tHi,\n\n\tI have a problem, I hope some of the 'gurus' can help me solve.\n\n\tBackground of the probl

### 9.  Now with dependent and independent data available for both train and test datasets, using TfidfVectorizer fit and transform the training data and test data and get the tfidf features for both

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:

vectorizer_tfidf_essay = TfidfVectorizer()
vectorizer_tfidf_essay.fit(twenty_train.data)

text_tfidf_train = vectorizer_tfidf_essay.transform(twenty_train.data)
print("Shape of matrix after one hot encoding ",text_tfidf_train.shape)

Shape of matrix after one hot encoding  (2257, 35788)


In [16]:
text_tfidf_test = vectorizer_tfidf_essay.transform(twenty_test.data)
print("Shape of matrix after one hot encoding ",text_tfidf_test.shape)

Shape of matrix after one hot encoding  (1502, 35788)


### 10. Use logisticRegression with tfidf features as input and targets as output and train the model and report the train and test accuracy score

In [0]:
from sklearn.linear_model import LogisticRegression

In [18]:
clf = LogisticRegression(random_state=7, solver='lbfgs').fit(text_tfidf_train, twenty_train.target)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [0]:
y_pred = clf.predict(text_tfidf_test)

In [21]:
clf.score(text_tfidf_test,twenty_test.target)

0.8861517976031957

In [22]:

clf.predict(text_tfidf_test)
clf.predict_proba(text_tfidf_test)

array([[0.13257859, 0.18671468, 0.5457058 , 0.13500093],
       [0.06860903, 0.06635982, 0.79761531, 0.06741585],
       [0.25231966, 0.11171898, 0.41232806, 0.2236333 ],
       ...,
       [0.15320491, 0.07887865, 0.51366741, 0.25424903],
       [0.09018272, 0.30742503, 0.51170009, 0.09069217],
       [0.06842466, 0.75212887, 0.13001014, 0.04943634]])